# a-8. ピボット

データ分析の世界で表構造のデータを扱う時、大きく分けて、ワイド型(wide format)とロング型(long format)(もしくは、スタック型(stacked format))と呼ばれる２つの表のフォーマットを意識しておくとよい。
これら２つの違いは、キー(値を唯一識別するための項目)を列名として持つか(ワイド型)、１つの列の値として持つか(ロング型)の違いである。
下図の左がワイド型で右がロング型の例である。いずれも「売上」という値を識別するためのキー項目として企業名(A〜E)とQuater(1Q〜4Q)があり、どちらの表も内容的には変わらず、フォーマットが違うだけである。

ロング型の利点は、キー項目を列で持っているために、`groupby()`などのキー単位の処理を伴う処理がやりやすいことにある。
一方でワイド型は、回帰モデルなどの入力データで利用されるフォーマットである(1行1サンプル)。
前処理はロング型で行い、モデル構築やデータ内容の表示といった分析の最終工程でワイド型が利用されるのが典型的なパターンである。

<img width="60%" src="./figures/pivot/wide_long.png">

---
## 1. pivot, stack, unstack(ワイド型-ロング型の変換)
## 2. pivot_table(集計を伴うpivot)
## 3. cross_table(件数集計)
## 4. get_dummies(ダミー変数の生成)
---
### 関連ドキュメント
- [Reshaping and pivot tables](https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html)

---
### APIリファレンス
- [DataFrame.pivot](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot.html)
- [DataFrame.stack](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.stack.html)
- [DataFrame.unstack](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.unstack.html)
- [DataFrame.pivot_table](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot_table.html)
- [pandas.unstack](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.unstack.html)
- [Series.str.get_dummies](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.get_dummies.html)

---

In [1]:
import numpy as np
import pandas as pd

---
# 1. pivot, stack, unstack

stack型とmatrix型の表構造を相互変換する。

### 入力データ

<pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
d = pd.DataFrame(
    {'r':['A', 'A', 'A', 'B', 'B', 'B'],
     'c':['x', 'y', 'z', 'x', 'y', 'z'], 
     'v':[40, 50, 60, 70, 80, 90]}
)
</pre>

<table border="1" style="table-layout:fixed;width:100%;">
    <colgroup>
      <col style="width:3%;">
      <col style="width:30%;">
      <col style="width:25%;">  
      <col style="width:25%;">
      <col style="width:17%;">
    </colgroup>
<tbody>

<tr><th align="left">no.</th><th align="left">実行コード</th><th align="left">入力</th><th align="left">出力</th><th align="left">備考</th></tr>
    
<tr style="background:#fff; border:1px solid #cc0000;">
<td>1-1</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
dd = d.pivot(index='r', columns='c', values='v')
print(dd)
</pre></td>
<td><img width="60%" src="./figures/pivot/pivot1-1data.png"></td>
<td><img width="60%" src="./figures/pivot/pivot1-1.png"></td>
<td>
<b>pivot()</b>は、stack形式のデータをmatrix形式のデータに変換する。indexで行ラベル、columnsで列ラベル、valuesで値となる列名を指定する。
ただし、同じ行ラベル-列ラベルがあればエラーとなることに注意する。
</td>
</tr>
   
<tr style="background:#fff; border:1px solid #cc0000;">
<td>1-2</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
# 1-1を実行した後に実行できる。
sr = dd.stack()
print(sr)
</pre></td>
<td><img width="60%" src="./figures/pivot/pivot1-2data.png"></td>
<td><img width="50%" src="./figures/pivot/pivot1-2.png"></td>
<td>
<b>stack()</b>は、全ての列名を値としてstack形式に変換する。ちょうどpivot()の逆の動きをする。
</td>
</tr>
     

<tr style="background:#fff; border:1px solid #cc0000;">
<td>1-3</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
# 1-2を実行した後に実行できる。
print(sr.unstack())
</pre></td>
<td><img width="50%" src="./figures/pivot/pivot1-3data.png"></td>
<td><img width="60%" src="./figures/pivot/pivot1-3.png"></td>
<td>
<b>unstack()</b>は、行のマルチインデックスの内側の列をpivot()のように列に展開する。
multi-indexの場合はunstack()を使うことでpivot()が実現できると考えればよい。
</td>
</tr>

<tr style="background:#fff; border:1px solid #cc0000;">
<td>1-4</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
# 1-2を実行した後に実行できる。
sr.name = 'v'
print(sr.reset_index())
</pre></td>
<td><img width="60%" src="./figures/pivot/pivot1-4data.png"></td>
<td><img width="60%" src="./figures/pivot/pivot1-4.png"></td>
<td>
結果srは行ラベルがマルチインデクスのSeriesなので、名前を設定してインデックスをresetすれば、完全に元のデータdに戻る。
</td>
</tr>

</tbody>
</table>

In [2]:
print('''
# 入力データ
>>> d = pd.DataFrame(
        {'r':['A', 'A', 'A', 'B', 'B', 'B'],
         'c':['x', 'y', 'z', 'x', 'y', 'z'], 
         'v':[40, 50, 60, 70, 80, 90]}
    )
>>> print(d)
''')
d = pd.DataFrame(
    {'r':['A', 'A', 'A', 'B', 'B', 'B'],
     'c':['x', 'y', 'z', 'x', 'y', 'z'], 
     'v':[40, 50, 60, 70, 80, 90]}
)
print(d)

print('''
# no.1-1
# pivot()は、stack形式のデータをmatrix形式のデータに変換する。
# indexで行ラベル、columnsで列ラベル、valuesで値となる列名を指定する。
# ただし、同じ行ラベル-列ラベルがあればエラーとなることに注意する。
>>> dd = d.pivot(index='r', columns='c', values='v')
>>> print(dd)''')
dd = d.pivot(index='r', columns='c', values='v')
print(dd)

print('''
# no.1-2
# stack()は、全ての列名を値としてstack形式に変換する。ちょうどpivot()の逆の動きをする。
>>> sr = dd.stack()
>>> print(sr)''')
sr = dd.stack()
print(sr)

print('''
# no.1-3
# unstack()は、行のマルチインデックスの内側の列をpivot()のように列に展開する。
# multi indexの場合はunstack()を使うことでpivot()が実現できると考えればよい。
>>> print(sr.unstack())''')
print(sr.unstack())


print('''
# no.1-4
# 結果srは行ラベルがマルチインデクスのSeriesなので、名前を設定してインデックスをresetすれば、完全に元のデータdに戻る。
>>> sr.name = 'v'
>>> print(sr.reset_index())''')
sr.name = 'v'
print(sr.reset_index())



# 入力データ
>>> d = pd.DataFrame(
        {'r':['A', 'A', 'A', 'B', 'B', 'B'],
         'c':['x', 'y', 'z', 'x', 'y', 'z'], 
         'v':[40, 50, 60, 70, 80, 90]}
    )
>>> print(d)

   r  c   v
0  A  x  40
1  A  y  50
2  A  z  60
3  B  x  70
4  B  y  80
5  B  z  90

# no.1-1
# pivot()は、stack形式のデータをmatrix形式のデータに変換する。
# indexで行ラベル、columnsで列ラベル、valuesで値となる列名を指定する。
# ただし、同じ行ラベル-列ラベルがあればエラーとなることに注意する。
>>> dd = d.pivot(index='r', columns='c', values='v')
>>> print(dd)
c   x   y   z
r            
A  40  50  60
B  70  80  90

# no.1-2
# stack()は、全ての列名を値としてstack形式に変換する。ちょうどpivot()の逆の動きをする。
>>> sr = dd.stack()
>>> print(sr)
r  c
A  x    40
   y    50
   z    60
B  x    70
   y    80
   z    90
dtype: int64

# no.1-3
# unstack()は、行のマルチインデックスの内側の列をpivot()のように列に展開する。
# multi indexの場合はunstack()を使うことでpivot()が実現できると考えればよい。
>>> print(sr.unstack())
c   x   y   z
r            
A  40  50  60
B  70  80  90

# no.1-4
# 結果srは行ラベルがマルチインデクスのSeriesなので、名前を設定してインデックスをresetすれば、完全に元のデータdに戻る。
>>> sr.name = 'v'
>>> pr

---
# 2. pivot_table
pivot()は表の構造を変換するだけであるのに対して、pivot_tableは値の集計を伴うpivotと考えればよい。

### 入力データ
pivot()で使った入力データとは、行(r)-列(r)の値に重複がある点が異なる。
A-xとB-yは２件存在する。
<pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
d = pd.DataFrame(
    {'r':['A', 'A', 'A', 'B', 'B', 'B', 'C'],
     'c':['x', 'x', 'y', 'x', 'y', 'y', 'y'], 
     'v':[40, 50, 60, 70, 80, 90, 100]}
)
</pre>

<table border="1" style="table-layout:fixed;width:100%;">
    <colgroup>
      <col style="width:3%;">
      <col style="width:30%;">
      <col style="width:25%;">  
      <col style="width:25%;">
      <col style="width:17%;">
    </colgroup>
<tbody>

<tr><th align="left">no.</th><th align="left">実行コード</th><th align="left">入力</th><th align="left">出力</th><th align="left">備考</th></tr>
 
<tr style="background:#fff; border:1px solid #cc0000;">
<td>2-1</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
dd = d.pivot_table(
    index='r',
    columns='c', 
    values='v',
    aggfunc=np.sum)
print(dd)
</pre></td>
<td><img width="80%" src="./figures/pivot/pivot2-1data.png"></td>
<td><img width="60%" src="./figures/pivot/pivot2-1.png"></td>
<td>
pivot()と同様に、stack形式のデータをmatrix形式のデータに変換し値を集計する。<br>
<b>index</b>で行ラベル、<b>columns</b>で列ラベル、<b>values</b>で値となる列名を指定する。
そして、同じ行ラベル-列ラベルの値をどのように集計するかを<b>aggfunc=</b>で指定する(デフォルトはnp.mean)。<br>
ここではnp.sum(合計)関数で集計している。matrixにしたときに、行ラベル-列ラベルの値が存在しなければNAとなる。
</td>
</tr>
 
<tr style="background:#fff; border:1px solid #cc0000;">
<td>2-2</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
dd = d.pivot_table(
    index='r',
    columns='c', 
    values='v', 
    aggfunc=np.sum, 
    fill_value=0)
print(dd)
</pre></td>
<td><img width="90%" src="./figures/pivot/pivot2-2data.png"></td>
<td><img width="60%" src="./figures/pivot/pivot2-2.png"></td>
<td>
<b>fill_value=</b>を指定すれば、NAがその値で出力される。
</td>
</tr>
 
<tr style="background:#fff; border:1px solid #cc0000;">
<td>2-3</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
dd = d.pivot_table(
    index='r', 
    columns='c',
    values='v', 
    aggfunc=np.sum,
    fill_value=0,
    margins=True)
print(dd)
</pre></td>
<td><img width="90%" src="./figures/pivot/pivot2-3data.png"></td>
<td><img width="70%" src="./figures/pivot/pivot2-3.png"></td>
<td>
<b>margins=True</b>とすれば周辺の集約結果が追加される。
</td>
</tr>
 
<tr style="background:#fff; border:1px solid #cc0000;">
<td>2-4</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
dd = d.pivot_table(
    index='r', 
    columns='c',
    values='v',
    aggfunc=['count', 'sum', np.mean, np.median, np.min, np.argmax], 
    fill_value=0)
print(dd)
</pre></td>
<td><img width="100%" src="./figures/pivot/pivot2-4data.png"></td>
<td><img width="100%" src="./figures/pivot/pivot2-4.png"></td>
<td>
複数の集約関数を指定すれば列がmulti-indexとなって全ての集約関数の結果が出力される。
</td>
</tr>
 
</tbody>
</table>

In [3]:
print('''
# 入力データ
# pivot()で使った入力データとは、行(r)-列(r)の値に重複がある点が異なる。
# A-xとB-yは２件存在する。
>>> d = pd.DataFrame(
        {'r':['A', 'A', 'A', 'B', 'B', 'B', 'C'],
         'c':['x', 'x', 'y', 'x', 'y', 'y', 'y'], 
         'v':[40, 50, 60, 70, 80, 90, 100]}
    )
>>> print(d)''')
d = pd.DataFrame(
    {'r':['A', 'A', 'A', 'B', 'B', 'B', 'C'],
     'c':['x', 'x', 'y', 'x', 'y', 'y', 'y'], 
     'v':[40, 50, 60, 70, 80, 90, 100]}
)
print(d)

print('''
# no.2-1
# pivot()と同様に、stack形式のデータをmatrix形式のデータに変換し値を集計する。
# indexで行ラベル、columnsで列ラベル、valuesで値となる列名を指定する。
# そして同じ行ラベル-列ラベルの値をどのように集計するかをaggfunc=で指定する(デフォルトはnp.mean)。
# 以下ではnp.sum(合計)関数で集計している。
# matrixにしたときに、行ラベル-列ラベルの値が存在しなければNAとなる。
>>> dd = d.pivot_table(
        index='r',
        columns='c', 
        values='v',
        aggfunc=np.sum)
>>> print(dd)''')
dd = d.pivot_table(
    index='r',
    columns='c', 
    values='v',
    aggfunc=np.sum)
print(dd)

print('''
# no.2-2
# fill_value=を指定すれば、NAがその値で出力される。
>>> dd = d.pivot_table(
        index='r',
        columns='c', 
        values='v', 
        aggfunc=np.sum, 
        fill_value=0)
>>> print(dd)''')
dd = d.pivot_table(
    index='r',
    columns='c', 
    values='v', 
    aggfunc=np.sum, 
    fill_value=0)
print(dd)

print('''
# no.2-3
# margins=Trueとすれば周辺の集約結果が追加される。
>>> dd = d.pivot_table(
        index='r', 
        columns='c',
        values='v', 
        aggfunc=np.sum,
        fill_value=0,
        margins=True)
print(dd)
>>> print(dd)''')
dd = d.pivot_table(
    index='r', 
    columns='c',
    values='v', 
    aggfunc=np.sum,
    fill_value=0,
    margins=True)
print(dd)

print('''
# no.2-4
# 複数の集約関数を指定すれば列がmulti-indexとなって全ての集約関数の結果が出力される。
>>> dd = d.pivot_table(
        index='r', 
        columns='c',
        values='v',
        aggfunc=['count', 'sum', np.mean, np.median, np.min, np.argmax], 
        fill_value=0)
>>> print(dd)''')
dd = d.pivot_table(
    index='r', 
    columns='c',
    values='v',
    aggfunc=['count', 'sum', np.mean, np.median, np.min, np.argmax], 
    fill_value=0)
print(dd)



# 入力データ
# pivot()で使った入力データとは、行(r)-列(r)の値に重複がある点が異なる。
# A-xとB-yは２件存在する。
>>> d = pd.DataFrame(
        {'r':['A', 'A', 'A', 'B', 'B', 'B', 'C'],
         'c':['x', 'x', 'y', 'x', 'y', 'y', 'y'], 
         'v':[40, 50, 60, 70, 80, 90, 100]}
    )
>>> print(d)
   r  c    v
0  A  x   40
1  A  x   50
2  A  y   60
3  B  x   70
4  B  y   80
5  B  y   90
6  C  y  100

# no.2-1
# pivot()と同様に、stack形式のデータをmatrix形式のデータに変換し値を集計する。
# indexで行ラベル、columnsで列ラベル、valuesで値となる列名を指定する。
# そして同じ行ラベル-列ラベルの値をどのように集計するかをaggfunc=で指定する(デフォルトはnp.mean)。
# 以下ではnp.sum(合計)関数で集計している。
# matrixにしたときに、行ラベル-列ラベルの値が存在しなければNAとなる。
>>> dd = d.pivot_table(
        index='r',
        columns='c', 
        values='v',
        aggfunc=np.sum)
>>> print(dd)
c     x      y
r             
A  90.0   60.0
B  70.0  170.0
C   NaN  100.0

# no.2-2
# fill_value=を指定すれば、NAがその値で出力される。
>>> dd = d.pivot_table(
        index='r',
        columns='c', 
        values='v', 
        aggfunc=np.sum, 
        fill_value=0)
>>> print(dd)
c   x    y
r   

---
# 3. cross_table
クロス集計表を作成する。stack型のデータから行-列の値の組み合わせの件数をカウントする。pivot()やpivot_table()と違い、pandasメソッドであり、行と列となる２つのSeriesをパラメータとして与える。

### 入力データ

<pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
d = pd.DataFrame(
    {'r':['A', 'A', 'A', 'B', 'B', 'B', 'C'],
     'c':['x', 'x', 'y', 'x', 'y', 'y', 'y'],
     'v':[40, 50, 60, 70, 80, 90, 100]}
)
</pre>

<table border="1" style="table-layout:fixed;width:100%;">
    <colgroup>
      <col style="width:3%;">
      <col style="width:30%;">
      <col style="width:25%;">  
      <col style="width:25%;">
      <col style="width:17%;">
    </colgroup>
<tbody>

<tr><th align="left">no.</th><th align="left">実行コード</th><th align="left">入力</th><th align="left">出力</th><th align="left">備考</th></tr>
   
<tr style="background:#fff; border:1px solid #cc0000;">
<td>3-1</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
dd = pd.crosstab(d['r'], d['c'])
print(dd)
</pre></td>
<td><img width="70%" src="./figures/pivot/pivot3-1data.png"></td>
<td><img width="60%" src="./figures/pivot/pivot3-1.png"></td>
<td>
行と列を表す２つのSeriesを与え、その件数をカウントする。
</td>
</tr>

<tr style="background:#fff; border:1px solid #cc0000;">
<td>3-2</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
dd = pd.crosstab(d['r'], d['c'], margins=True)
print(dd)
</pre></td>
<td><img width="90%" src="./figures/pivot/pivot3-2data.png"></td>
<td><img width="60%" src="./figures/pivot/pivot3-2.png"></td>
<td>
<b>margins=True</b>で行と列の件数合計を出力する。
</td>
</tr>

<tr style="background:#fff; border:1px solid #cc0000;">
<td>3-3</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
dd = pd.crosstab(d['r'], d['c'], normalize=True)
print(dd)
</pre></td>
<td><img width="90%" src="./figures/pivot/pivot3-3data.png"></td>
<td><img width="70%" src="./figures/pivot/pivot3-3.png"></td>
<td>
<b>normalize=True</b>にて全体に対する構成比を出力する。
</td>
</tr>

<tr style="background:#fff; border:1px solid #cc0000;">
<td>3-4</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
dd = pd.crosstab(d['r'], d['c'], normalize='index')
print(dd)
</pre></td>
<td><img width="90%" src="./figures/pivot/pivot3-4data.png"></td>
<td><img width="80%" src="./figures/pivot/pivot3-4.png"></td>
<td>
<b>normalize='index'</b>で行に対する構成比となる。
</td>
</tr>

<tr style="background:#fff; border:1px solid #cc0000;">
<td>3-5</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
dd = pd.crosstab(d['r'], d['c'], normalize='columns')
print(dd)
</pre></td>
<td><img width="90%" src="./figures/pivot/pivot3-5data.png"></td>
<td><img width="70%" src="./figures/pivot/pivot3-5.png"></td>
<td>
<b>normalize='columns'</b>にて列に対する構成比を出力する。
</td>
</tr>

<tr style="background:#fff; border:1px solid #cc0000;">
<td>3-6</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
dd = pd.crosstab(d['r'], d['c'], values=d['v'], aggfunc=np.sum)
print(dd)
</pre></td>
<td><img width="100%" src="./figures/pivot/pivot3-6data.png"></td>
<td><img width="70%" src="./figures/pivot/pivot3-6.png"></td>
<td>
<b>values=</b>と<b>aggfunc=</b>を指定することでpivot_table()と同様にvaluesの列を集約して出力する。
</td>
</tr>
 
</tbody>
</table>

In [4]:
print('''
# 入力データ
# pivot_table()で使った入力データと同じ。
>>> d = pd.DataFrame(
        {'r':['A', 'A', 'A', 'B', 'B', 'B', 'C'],
         'c':['x', 'x', 'y', 'x', 'y', 'y', 'y'],
         'v':[40, 50, 60, 70, 80, 90, 100]}
    )
>>> print(d)''')
d = pd.DataFrame(
    {'r':['A', 'A', 'A', 'B', 'B', 'B', 'C'],
     'c':['x', 'x', 'y', 'x', 'y', 'y', 'y'],
     'v':[40, 50, 60, 70, 80, 90, 100]}
)
print(d)

print('''
# no.3-1
# 行と列を表す２つのSeriesを与え、その件数をカウントする。
>>> dd = pd.crosstab(d['r'], d['c'])
>>> print(dd)''')
dd = pd.crosstab(d['r'], d['c'])
print(dd)

print('''
# no.3-2
# margins=Trueで行と列の件数合計を出力する。
>>> dd = pd.crosstab(d['r'], d['c'], margins=True)
>>> print(dd)''')
dd = pd.crosstab(d['r'], d['c'], margins=True)
print(dd)

print('''
# no.3-3
# normalize=Trueにて全体に対する構成比を出力する。
>>> dd = pd.crosstab(d['r'], d['c'], normalize=True)
>>> print(dd)''')
dd = pd.crosstab(d['r'], d['c'], normalize=True)
print(dd)


print('''
# no.3-4
# normalize='index'で行に対する構成比となる。
>>> dd = pd.crosstab(d['r'], d['c'], normalize='index')
>>> print(dd)''')
dd = pd.crosstab(d['r'], d['c'], normalize='index')
print(dd)

print('''
# no.3-5
# normalize='columns'にて列に対する構成比を出力する。
>>> dd = pd.crosstab(d['r'], d['c'], normalize='columns')
>>> print(dd)''')
dd = pd.crosstab(d['r'], d['c'], normalize='columns')
print(dd)

print('''
# no.3-6
# values=とaggfunc=を指定することでpivot_table()と同様にvaluesの列を集約して出力する。
>>> dd = pd.crosstab(d['r'], d['c'], values=d['v'], aggfunc=np.sum)
>>> print(dd)''')
dd = pd.crosstab(d['r'], d['c'], values=d['v'], aggfunc=np.sum)
print(dd)



# 入力データ
# pivot_table()で使った入力データと同じ。
>>> d = pd.DataFrame(
        {'r':['A', 'A', 'A', 'B', 'B', 'B', 'C'],
         'c':['x', 'x', 'y', 'x', 'y', 'y', 'y'],
         'v':[40, 50, 60, 70, 80, 90, 100]}
    )
>>> print(d)
   r  c    v
0  A  x   40
1  A  x   50
2  A  y   60
3  B  x   70
4  B  y   80
5  B  y   90
6  C  y  100

# no.3-1
# 行と列を表す２つのSeriesを与え、その件数をカウントする。
>>> dd = pd.crosstab(d['r'], d['c'])
>>> print(dd)
c  x  y
r      
A  2  1
B  1  2
C  0  1

# no.3-2
# margins=Trueで行と列の件数合計を出力する。
>>> dd = pd.crosstab(d['r'], d['c'], margins=True)
>>> print(dd)
c    x  y  All
r             
A    2  1    3
B    1  2    3
C    0  1    1
All  3  4    7

# no.3-3
# normalize=Trueにて全体に対する構成比を出力する。
>>> dd = pd.crosstab(d['r'], d['c'], normalize=True)
>>> print(dd)
c         x         y
r                    
A  0.285714  0.142857
B  0.142857  0.285714
C  0.000000  0.142857

# no.3-4
# normalize='index'で行に対する構成比となる。
>>> dd = pd.crosstab(d['r'], d['c'], normalize='index')
>>> print(dd)
c        

---
# 4. get_dummies

### 入力データ

<pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
d = pd.DataFrame(
    {'r':['A', 'A', 'A', 'B', 'B', 'B', 'C'], 
     'c':['x', 'x', 'y', 'x', 'y', 'y', 'y'], 
     'v':[40, 50, 60, 70, 80, 90, 100]}
)
</pre>

<table border="1" style="table-layout:fixed;width:100%;">
    <colgroup>
      <col style="width:3%;">
      <col style="width:30%;">
      <col style="width:25%;">  
      <col style="width:25%;">
      <col style="width:17%;">
    </colgroup>
<tbody>

<tr><th align="left">no.</th><th align="left">実行コード</th><th align="left">入力</th><th align="left">出力</th><th>備考</th></tr>
    
<tr style="background:#fff; border:1px solid #cc0000;">
<td>4-1</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
dd = pd.get_dummies(d['r'])
print(dd)
</pre></td>
<td><img width="60%" src="./figures/pivot/pivot4-1data.png"></td>
<td><img width="60%" src="./figures/pivot/pivot4-1.png"></td>
<td>
r列の値A,B,Cをdummy化する。A,B,Cを列にして、それぞれの値の行は1に、それ以外は0を出力する。
</td>
</tr>
 
<tr style="background:#fff; border:1px solid #cc0000;">
<td>4-2</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
dd = pd.get_dummies(d['r'], prefix='r')
print(dd)
</pre></td>
<td><img width="60%" src="./figures/pivot/pivot4-2data.png"></td>
<td><img width="70%" src="./figures/pivot/pivot4-2.png"></td>
<td>
<b>prefix=</b>を指定することで、その文字列が列名の頭に追加される。
</td>
</tr>

<tr style="background:#fff; border:1px solid #cc0000;">
<td>4-3</td>
<td><pre style="overflow-x:auto;white-space:pre;padding: 1em; border-radius: 5px; background: #eeeeee">
dd = pd.get_dummies(pd.cut(d['v'], [30, 60, 80]))
print(dd)
</pre></td>
<td><img width="60%" src="./figures/pivot/pivot4-3data.png"></td>
<td><img width="70%" src="./figures/pivot/pivot4-3.png"></td>
<td>
<b>pd.cut()</b>と組み合わせることで、数値変数をdummy化できる。
</td>
</tr>

</tbody>
</table>

In [5]:
print('''
# 入力データ
>>> d = pd.DataFrame(
    {'r':['A', 'A', 'A', 'B', 'B', 'B', 'C'], 
     'c':['x', 'x', 'y', 'x', 'y', 'y', 'y'], 
     'v':[40, 50, 60, 70, 80, 90, 100]}
)
>>> print(d)''')
d = pd.DataFrame(
    {'r':['A', 'A', 'A', 'B', 'B', 'B', 'C'], 
     'c':['x', 'x', 'y', 'x', 'y', 'y', 'y'], 
     'v':[40, 50, 60, 70, 80, 90, 100]}
)
print(d)

print('''
# no.4-1
# r列の値A,B,Cをdummy化する。
# A,B,Cを列にして、それぞれの値の行は1に、それ以外は0を出力する。
>>> dd = pd.get_dummies(d['r'])
>>> print(dd)''')
dd = pd.get_dummies(d['r'])
print(dd)

print('''
# no.4-2
# prefix=を指定することで、その文字列が列名の頭に追加される。
>>> dd = pd.get_dummies(d['r'], prefix='r')
>>> print(dd)''')
dd = pd.get_dummies(d['r'], prefix='r')
print(dd)

print('''
# no.4-3
# pd.cut()と組み合わせることで、数値変数をdummy化できる。
>>> dd = pd.get_dummies(pd.cut(d['v'], [30, 60, 80]))
>>> print(dd)''')
dd = pd.get_dummies(pd.cut(d['v'], [30, 60, 80]))
print(dd)



# 入力データ
>>> d = pd.DataFrame(
    {'r':['A', 'A', 'A', 'B', 'B', 'B', 'C'], 
     'c':['x', 'x', 'y', 'x', 'y', 'y', 'y'], 
     'v':[40, 50, 60, 70, 80, 90, 100]}
)
>>> print(d)
   r  c    v
0  A  x   40
1  A  x   50
2  A  y   60
3  B  x   70
4  B  y   80
5  B  y   90
6  C  y  100

# no.4-1
# r列の値A,B,Cをdummy化する。
# A,B,Cを列にして、それぞれの値の行は1に、それ以外は0を出力する。
>>> dd = pd.get_dummies(d['r'])
>>> print(dd)
   A  B  C
0  1  0  0
1  1  0  0
2  1  0  0
3  0  1  0
4  0  1  0
5  0  1  0
6  0  0  1

# no.4-2
# prefix=を指定することで、その文字列が列名の頭に追加される。
>>> dd = pd.get_dummies(d['r'], prefix='r')
>>> print(dd)
   r_A  r_B  r_C
0    1    0    0
1    1    0    0
2    1    0    0
3    0    1    0
4    0    1    0
5    0    1    0
6    0    0    1

# no.4-3
# pd.cut()と組み合わせることで、数値変数をdummy化できる。
>>> dd = pd.get_dummies(pd.cut(d['v'], [30, 60, 80]))
>>> print(dd)
   (30, 60]  (60, 80]
0         1         0
1         1         0
2         1         0
3         0         1
4         0         1
5         0         0
6  